In [27]:
import ee
from osgeo import gdal, osr

ee.Authenticate()
ee.Initialize(project="ee-start-hack-2025")
collection = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY_AGGR")

count = collection.size().getInfo()
print(count)

901


In [28]:
import urllib.request
import os

In [29]:
tif_path = 'temp_diff/Gridded_Population_Density_Data/Assaba_Pop_2020_norm.tif_diff.tif'
ds = gdal.Open(tif_path)

geo_transform = ds.GetGeoTransform()
x_origin    = geo_transform[0]     # top left x-coordinate
pixel_width = geo_transform[1]     # pixel width
y_origin    = geo_transform[3]     # top left y-coordinate
pixel_height= geo_transform[5]     # pixel height (often negative)

cols = ds.RasterXSize
rows = ds.RasterYSize

xmin = x_origin
xmax = x_origin + pixel_width * cols
ymax = y_origin
ymin = y_origin + pixel_height * rows

print(f"Native Bounds:\n xmin: {xmin}, ymin: {ymin}, xmax: {xmax}, ymax: {ymax}")

Native Bounds:
 xmin: -12.849997509270906, ymin: 15.19905582311569, xmax: -10.651860199739515, ymax: 18.29999952763319


In [30]:
aoi = ee.Geometry.Rectangle([xmin, ymin, xmax, ymax])
print(aoi.getInfo())

{'type': 'Polygon', 'coordinates': [[[-12.849997509270906, 15.19905582311569], [-10.651860199739515, 15.19905582311569], [-10.651860199739515, 18.29999952763319], [-12.849997509270906, 18.29999952763319], [-12.849997509270906, 15.19905582311569]]]}


In [31]:
start_date = '2020-01-01'
end_date = '2020-12-31'

collection = collection.filterDate(start_date, end_date)
count = collection.size().getInfo()

image_list = collection.toList(count)

In [32]:
download_dir = "test_dir_earth"
os.makedirs(download_dir, exist_ok=True)

In [38]:
# Assume images_list is your ee.List of images and count is its length.
for i in range(count):
    # Get image i as an ee.Image object.
    image = ee.Image(image_list.get(i))
    
    # Retrieve the date from the image property 'system:time_start' and format it.
    time_start = image.get('system:time_start')
    date_string = ee.Date(time_start).format('YYYY_MM_dd').getInfo()
    
    # Construct the local filename with the date included.
    file_name = os.path.join(download_dir, f"era5_mean_2m_air_temperature_{date_string}.tif")
    print(f"\nExporting image dated {date_string}")
    
    # Define export parameters, plugging in the AOI.
    export_params = {
        'scale': 10000,              # Pixel resolution in meters; adjust as needed.
        'crs': 'EPSG:4326',         # Coordinate reference system.
        'region': aoi,              # Your geographic area (AOI) defined above.
        'fileFormat': 'GeoTIFF'     # Output file format.
    }
    
    try:
        # Get the download URL.
        download_url = image.getDownloadURL(export_params)
        print("Download URL:", download_url)
        
        # Download the GeoTIFF from the URL.
        urllib.request.urlretrieve(download_url, file_name)
        print(f"Saved file: {file_name}")
    except Exception as e:
        print(f"Failed to export or download image for {date_string}: {e}")



Exporting image dated 2020_01_01
Download URL: https://earthengine.googleapis.com/v1/projects/ee-start-hack-2025/thumbnails/57048941f7b0e7028bbaa1bade9af328-281eed14259dff4f889f3deb3e804073:getPixels
Saved file: test_dir_earth/era5_mean_2m_air_temperature_2020_01_01.tif

Exporting image dated 2020_02_01
Download URL: https://earthengine.googleapis.com/v1/projects/ee-start-hack-2025/thumbnails/59b67fa74259451657236cc5830cf6e0-d16b12b67f31ff8df2a502629b8efd5d:getPixels
Saved file: test_dir_earth/era5_mean_2m_air_temperature_2020_02_01.tif

Exporting image dated 2020_03_01
Download URL: https://earthengine.googleapis.com/v1/projects/ee-start-hack-2025/thumbnails/29fe1ed68bfbfdb61566dc8987a2672b-3b4682d76e6c12f206dd953083094012:getPixels
Saved file: test_dir_earth/era5_mean_2m_air_temperature_2020_03_01.tif

Exporting image dated 2020_04_01
Download URL: https://earthengine.googleapis.com/v1/projects/ee-start-hack-2025/thumbnails/e8119ecffe4afe881be82e6d5f268cb5-3bb7aa7d1209b8e501ff5dd18e

In [39]:
import os
from osgeo import gdal
import matplotlib.pyplot as plt
import numpy as np

# Specify the single GeoTIFF file you want to visualize.
tif_file = "test_dir_earth/era5_mean_2m_air_temperature_2020_01_01.tif"

# Choose the specific band (1-indexed, so band 1 is the first band)
specific_band = 'temperature_2m'  # Adjust this number as needed

# Open the GeoTIFF using GDAL.
dataset = gdal.Open(tif_file)
if dataset is None:
    raise RuntimeError("Unable to open the GeoTIFF file.")

# Check whether the selected band is within the image's band count.
num_bands = dataset.RasterCount
if specific_band < 1 or specific_band > num_bands:
    raise ValueError(f"Selected band {specific_band} is out of range. This image has only {num_bands} band(s).")

print(f"Visualizing file '{os.path.basename(tif_file)}', Band {specific_band}")

# Read the specific band data.
band = dataset.GetRasterBand(specific_band)
arr = band.ReadAsArray()

# Optional: Normalize the array for display (improves contrast if pixel values have a wide range)
def normalize_array(arr):
    arr = arr.astype(np.float32)
    p2, p98 = np.percentile(arr, (2, 98))
    return np.clip((arr - p2) / (p98 - p2), 0, 1)

arr_normalized = normalize_array(arr)

# Visualize the specific band.
plt.figure(figsize=(8, 6))
plt.imshow(arr_normalized, cmap='viridis')  # Choose a cmap, e.g., 'viridis' or 'gray'
plt.title(f"{os.path.basename(tif_file)} - Band {specific_band}")
plt.colorbar(label="Pixel value")
plt.axis("off")
plt.show()


ERROR 4: `test_dir_earth/era5_mean_2m_air_temperature_2020_01_01.tif' not recognized as being in a supported file format.


RuntimeError: Unable to open the GeoTIFF file.

In [3]:
start_date = '2010-01-01'
end_date = '2010-12-31'
collection_filtered = collection.filterDate(start_date, end_date)

In [4]:
aoi = ee.Geometry.Rectangle([-12.849997509270906, 15.19999948143959, -10.649997476488352, 18.29999952763319])

In [5]:
collection = (ee.ImageCollection('ECMWF/ERA5/MONTHLY')
              .filterBounds(aoi)
              .filterDate(start_date, end_date)
              .select('mean_2m_air_temperature'))  # Select the temperature_2m band

In [6]:
count = collection.size().getInfo()
print("Number of monthly images found:", count)

Number of monthly images found: 12


In [7]:
images_list = collection.toList(count)

In [8]:
download_dir = 'era5_downloads'
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

In [9]:
for i in range(count):
    # Get image i as an ee.Image object.
    image = ee.Image(images_list.get(i))
    
    # Retrieve the date from the image property 'system:time_start' and format it.
    # This property holds the timestamp for the image.
    time_start = image.get('system:time_start')
    date_string = ee.Date(time_start).format('YYYY_MM_dd').getInfo()
    
    # Construct the local filename with the date included.
    file_name = os.path.join(download_dir, f"era5_mean_2m_air_temperature_{date_string}.tif")
    print(f"\nExporting image dated {date_string}")
    
    # Define export parameters.
    export_params = {
        'scale': 1000,              # Pixel resolution in meters; adjust as needed.
        'crs': 'EPSG:4326',         # Coordinate reference system.
        'region': aoi,              # Your geographic area (AOI).
        'fileFormat': 'GeoTIFF'     # Output file format.
    }
    
    try:
        # Get the download URL.
        download_url = image.getDownloadURL(export_params)
        print("Download URL:", download_url)
        
        # Download the GeoTIFF from the URL.
        urllib.request.urlretrieve(download_url, file_name)
        print(f"Saved file: {file_name}")
    except Exception as e:
        print(f"Failed to export or download image for {date_string}: {e}")


Exporting image dated 2010_01_01
Download URL: https://earthengine.googleapis.com/v1/projects/ee-start-hack-2025/thumbnails/f47f64ee5662df8bd7d01151e818ab04-ce576f1d0720b6dfb4bcbcb8e2bad806:getPixels
Saved file: era5_downloads/era5_mean_2m_air_temperature_2010_01_01.tif

Exporting image dated 2010_02_01
Download URL: https://earthengine.googleapis.com/v1/projects/ee-start-hack-2025/thumbnails/290b4cc6f9c8f72a9f2666803ec062a2-350946c8bf7c2cc10b2fa0a9b91dc8ba:getPixels
Saved file: era5_downloads/era5_mean_2m_air_temperature_2010_02_01.tif

Exporting image dated 2010_03_01
Download URL: https://earthengine.googleapis.com/v1/projects/ee-start-hack-2025/thumbnails/6e9361f0955afe9e54e629669bf7c8c1-62b45ff227901c8b266832532b0aedeb:getPixels
Saved file: era5_downloads/era5_mean_2m_air_temperature_2010_03_01.tif

Exporting image dated 2010_04_01
Download URL: https://earthengine.googleapis.com/v1/projects/ee-start-hack-2025/thumbnails/4deae7b719811f4cc3828af44371381c-be2220e1dea6f73ac4ab5c426e

In [54]:
url = "era5_downloads/era5_mean_2m_air_temperature_2010_01_01.tif"
image = ee.Image(url)

In [55]:
import folium

# Define visualization parameters
vis_params = {
    'min': 13000,  # Minimum value for visualization
    'max': 16500,  # Maximum value for visualization
    'palette': ['blue', 'green', 'red']  # Color palette
}

# Get a URL for the image tile layer
map_id_dict = image.getMapId(vis_params)
tile_url = map_id_dict['tile_fetcher'].url_format

# Create a folium map
map_center = [0, 0]  # Latitude, Longitude for the map center
m = folium.Map(location=map_center, zoom_start=2)

# Add the Earth Engine image as a tile layer
folium.TileLayer(
    tiles=tile_url,
    attr='Google Earth Engine',
    overlay=True,
    name='MODIS LST'
).add_to(m)

# Add a layer control panel
folium.LayerControl().add_to(m)

# Display the map
m

EEException: Image.load: Image asset 'era5_downloads/era5_mean_2m_air_temperature_2010_01_01.tif' not found (does not exist or caller does not have access).

In [44]:
import requests
import rasterio

url = "https://earthengine.googleapis.com/v1/projects/ee-start-hack-2025/thumbnails/3f06bf1c8e7e32f938a9c689a441f72a-7dc356db53dc2936804427a118019f11:getPixels"
local_file = "downloaded_file.tif"

response = requests.get(url)
if response.status_code == 200:
    with open(local_file, "wb") as f:
        f.write(response.content)
else:
    raise Exception(f"Failed to download file: {response.status_code}")

# Open the downloaded file
with rasterio.open(local_file) as src:
    data = src.read(1)
    # Process the data...

RasterioIOError: 'downloaded_file.tif' not recognized as a supported file format.

In [38]:
import rasterio
from rasterio.plot import show

import matplotlib.pyplot as plt

# Open the GeoTIFF file from the URL
with rasterio.open(download_url) as src:
    # Read the data
    data = src.read(1)  # Read the first band

    # Plot the data
    plt.figure(figsize=(10, 8))
    show(data, transform=src.transform, cmap='viridis')
    plt.title('Mean 2m Air Temperature')
    plt.colorbar(plt.cm.ScalarMappable(cmap='viridis'), label='Temperature (K)')
    plt.show()

RasterioIOError: '/vsicurl/https://earthengine.googleapis.com/v1/projects/ee-start-hack-2025/thumbnails/3f06bf1c8e7e32f938a9c689a441f72a-7dc356db53dc2936804427a118019f11:getPixels' not recognized as a supported file format.

In [39]:
from osgeo import gdal

# Open one of the downloaded GeoTIFF files.
dataset = gdal.Open('era5_downloads/era5_temperature_2m_2021_01_01.tif')
if dataset is None:
    raise Exception("Failed to load the GeoTIFF file.")

# Retrieve basic information.
cols = dataset.RasterXSize
rows = dataset.RasterYSize
geotransform = dataset.GetGeoTransform()
projection = dataset.GetProjection()

print("Raster dimensions:", cols, "columns x", rows, "rows")
print("GeoTransform:", geotransform)
print("Projection:", projection)

# Read the first band into an array.
band = dataset.GetRasterBand(1)
data_array = band.ReadAsArray()

print("Data type:", data_array.dtype)
print("Data shape:", data_array.shape)

/root/miniforge3/envs/Hack25/lib/python3.13/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
ERROR 4: era5_downloads/era5_temperature_2m_2021_01_01.tif: No such file or directory


Exception: Failed to load the GeoTIFF file.